<a href="https://colab.research.google.com/github/innko-t/other/blob/main/trackmate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**trackmateの結果をtrack_idごとに座標、輝度と平均変位を出力。**

In [32]:
#@title 取得したいTRACK_IDを選択してください。 { display-mode: "form" }

import pandas as pd
import numpy as np
import io
import os
from google.colab import files
import ipywidgets as widgets
import shutil
!rm result.zip
shutil.rmtree('./result')

def check_track_id(file, file_name):
  input_csv = pd.read_csv(io.BytesIO(file[file_name]))
  req = input_csv.drop(range(3))
  check_ids = list(req["TRACK_ID"].unique())
  print('TRACK_IDチェック画面')
  checkboxes = [widgets.Checkbox(value=False, description=label) for label in check_ids]
  output = widgets.VBox(children=checkboxes)
  display(output)
  s = 'no'
  while s != 'ok':
    s = input('必要なTRACK_IDをチェックの完了が終了したら、"ok"を押してください\n')

  return req, checkboxes


uploaded = files.upload()
file_name = list(uploaded.keys())[0]
req, checkboxes = check_track_id(uploaded, file_name)


rm: cannot remove 'result.zip': No such file or directory


Saving track.csv to track.csv
TRACK_IDチェック画面


必要なTRACK_IDをチェックの完了が終了したら、"ok"を押してください
ok


In [33]:
#@title 取得したいTRACK_IDを選択して、"ok"を入力したら、左の再生ボタンをクリックしてください。 { display-mode: "form" }


selected_data = []
for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        selected_data = selected_data + [checkboxes[i].description]


def output_csv(req, selected_ids):
  total_number = int(input("観察したステップ数を入力してください：　\n"))
  ave_number = int(input("平均を取りたいステップ数を入力してください：　\n"))
  if total_number % ave_number != 0:
    print('観察したステップ数を平均を取りたいステップ数で割り切れる値にしてください')
    ave_number = int(input("平均を取りたいステップ数を入力してください：　\n"))
  else:
    t_l = list(range(0, total_number, ave_number))
    ave_df = pd.DataFrame(index=t_l)
    for check_id in selected_data:
      displacement = []
      check_df = pd.DataFrame(req[req["TRACK_ID"] == check_id][["POSITION_X", "POSITION_Y", "QUALITY", "FRAME"]])
      check_df["POSITION_X"] = check_df["POSITION_X"].astype(np.float32)
      check_df["POSITION_Y"] = check_df["POSITION_Y"].astype(np.float32)
      check_df["QUALITY"] = check_df["QUALITY"].astype(np.float32)
      check_df["FRAME"] = check_df["FRAME"].astype(np.int32)
      check_df = check_df.sort_values("FRAME")
      print(f"check_id:{check_id}の穴埋め前\n")
      print(check_df.info())

      n_x = check_df.iat[0, 0]
      n_y = check_df.iat[0, 1]
      if 0 not in check_df["FRAME"].values:
          check_df = check_df.append({'POSITION_X': n_x, 'POSITION_Y': n_y, 'FRAME': 0}, ignore_index=True)
      for i in range(1, total_number):
          if i in check_df['FRAME'].values:
              n_x = check_df[check_df['FRAME'] == i]['POSITION_X'].values[0]
              n_y = check_df[check_df['FRAME'] == i]['POSITION_Y'].values[0]
          else:
              check_df = check_df.append({'POSITION_X': n_x, 'POSITION_Y': n_y, 'FRAME': i}, ignore_index=True)

          displacement.append(((check_df.iat[i, 0] - check_df.iat[i - 1, 0]) ** 2 + (
                      check_df.iat[i, 1] - check_df.iat[i - 1, 1]) ** 2) ** 0.5)
      displacement.insert(0, 0)
      check_df["displacement"] = displacement
      check_df["FRAME"] = check_df["FRAME"].astype(np.int32)
      check_df = check_df.set_index('FRAME')
      check_df = check_df.sort_index()

      n_ave = []
      for t in t_l:
          n_ave.append(check_df[t:t + ave_number]["displacement"].mean())
      ave_column = pd.Series(n_ave, index=t_l, name="ID{}の平均変位".format(check_id))
      ave_df = pd.merge(ave_df, ave_column, how='outer', left_index=True, right_index=True)
      check_df.to_csv(f"./result/output_{check_id}.csv", encoding='shift-jis')

    ave_df.to_csv(f"./result/ave_{ave_number}-{total_number}.csv", encoding='shift-jis')


print('選択したTRACK_ID', selected_data)
!mkdir result
output_csv(req, selected_data)
!zip -r result.zip result
files.download("result.zip")
!rm {file_name}


選択したTRACK_ID ['1', '33']
観察したステップ数を入力してください：　
750
平均を取りたいステップ数を入力してください：　
3
check_id:1の穴埋め前

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 609 to 275
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   POSITION_X  750 non-null    float32
 1   POSITION_Y  750 non-null    float32
 2   QUALITY     750 non-null    float32
 3   FRAME       750 non-null    int32  
dtypes: float32(3), int32(1)
memory usage: 17.6 KB
None
check_id:33の穴埋め前

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 1998 to 1547
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   POSITION_X  714 non-null    float32
 1   POSITION_Y  714 non-null    float32
 2   QUALITY     714 non-null    float32
 3   FRAME       714 non-null    int32  
dtypes: float32(3), int32(1)
memory usage: 16.7 KB
None
  adding: result/ (stored 0%)
  adding: result/output_1.csv (deflated 5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>